# Décomposition LU d'une matrice $n\times n$

L'objectif de cette fiche est de vous aider à comprendre les étapes de la factorisation LU. Les différentes étapes sont détaillées à l'aide de codes Python qui permettent de les exécuter.

La section qui suit définit quelques fonctions Python bien pratiques... inutile de la lire 😄.


## 0. Préliminaire Python

In [1]:
from numpy.matlib import int32, float32
from numpy.linalg import inv, det, solve
from numpy.matlib import identity, mat, zeros
from random import randint
from IPython.display import display, HTML

def z(x):
    return abs(x)<1e-6

def nz(x):
    return not z(x)

def prod(M, *MS):
    R = M
    for X in MS:
        R = R * X
    return R

def rmat(n):
    "génère une matrix [n]x[n] décomposable LU entière"
    while True:
        L = identity(n,float)
        for i in range(n):
            for j in range(i):
                L[i,j] = randint(1,10)
        U = identity(n,float)
        for i in range(n):
            for j in range(i,n):
                U[i,j] = randint(1,10)
        A = L*U
        if nz(det(A)):
            return L*U
    
def latex(A):
    if type(A) is str:
        return A
    if type(A) is list:
        return r"\underbrace{" + r"\times ".join(map(latex,A)) + r"}_{\displaystyle" + latex(prod(*A)) + r"}"
    n=A.shape[0]
    s=r"\begin{pmatrix}"
    c=[]
    for i in range(n):
        l=[]
        for j in range(n):
            l.append("%g" % A[i,j])
        c.append('&'.join(l))
    return s+r"\\".join(c)+r"\end{pmatrix}"
    
def showlatex(s):
    display(HTML(r"$$" + s + r"$$"))
    
def show(A, *BS, online=False):
    if not BS:
        showlatex(latex(A))
    else:
        showlatex(r"\begin{eqnarray}" + latex(A) + r"&=&" + (r"=" if online else r"\\&=&").join([ r"\times ".join(map(latex,B)) for B in BS ]) + r"\end{eqnarray}")
        

## 1. Principe de la décomposition $LU$

Une matrice $A$ est décomposable en une factorisation $A = LU$ s'il existe une matrice triangulaire inférieure $L$ avec uniquement la valeur $1$ sur la diagonale et une matrice triangulaire supérieure $U$ telles que $A = LU$. Lorsqu'elle existe et que $A$ est inversible, cette factorisation est unique.

Prenons une telle matrice $A$ de taille $n\times n$ (le choix de $A$ change si on réexécute cette cellule) :

In [2]:
A=rmat(4)
show("A",[A])

Pour calculer les matrices $L$ et $U$, on peut procéder séquentiellement en construisant une suite de paires de matrices $(L_i,U_i)$ pour $i$ allant de $0$ à $n-1$ de sorte qu'à chaque étape $A = L_i\times U_i$. 

On part de la paire $L_0=\operatorname{Id}$ et $U_0=A$ jusqu'à arriver à $L_{n-1}=L$ et $U_{n-1}=U$. 

Toutes les matrices $L_i$ sont triangulaires inférieures avec uniquement la valeur $1$ sur la diagonale.

À l'étape $i$, on construit $L_i$ et $U_i$ à partir de $L_{i-1}$ et $U_{i-1}$ de sorte que les $i$ premières colonnes de $U_{i-1}$ soient en forme triangulaire supérieure (la colonne $i$ porte des $0$ pour les indices supérieurs à $i$). Pour cela, on annule successivement les $n-i$ éléments de la colonne $i$ à l'aide du pivot de Gauss et de la remarque suivante. Si $A = L_{i-1} U_{i-1}$ et que $P$ est une matrice inversible alors on a :
$$\begin{eqnarray}
A &=& L_{i-1} \times U_{i-1} \\
A &=& L_{i-1} \times \operatorname{Id} \times U_{i-1} \\
A &=& L_{i-1} \times \left( P^{-1} \times P \right) \times U_{i-1} \\
A &=& \left( L_{i-1} \times P^{-1} \right) \times \left( P  \times U_{i-1} \right) \\
\end{eqnarray}$$

En choisissant une matrice $P$, produit de pivots, qui nettoie la $i$ème colonne de $U_{i-1}$ et en supposant que $P^{-1}$ préserve les propriétés attendues pour $L_i$, c'est gagné !

## 2. Un pivot, des pivots

Notons $P_{l_i \leftarrow l_i + \alpha l_j}$ la matrice pivot qui lorsqu'on la multiplie à gauche d'une matrice $M$ produit la matrice $M'$ identique à $M$ sauf pour sa $i$ème ligne qui est la $i$ème ligne de $M$ à laquelle on a ajouté $\alpha$ fois sa $j$ème ligne. Il s'agit d'une matrice identité dont on modifie l'élément en position $(j,i)$ pour qu'il soit égal à $\alpha$. Voici un exemple :

In [3]:
def pivot(n, k, i, j):
    """Matrice [n]x[n] qui ajoute [k] fois la ligne [i] à la ligne [j]"""
    M = identity(n, float)
    M[j,i] = k
    return M

P = pivot(4,3,0,1)
show(r"\operatorname{P}_{l_1 \leftarrow l_1 + 3 l_0}", [P])
M = rmat(4)
show(P*M, [P, M])

Ce paramètre $k$ peut être défini afin de générer un 0 dans le résultat. Par exemple si on choisit $k=-\frac{M_{10}}{M_{00}}$ ou $k=-\frac{M_{20}}{M_{00}}$

In [4]:
M = rmat(4)
show(M)
a = -M[1,0]/M[0,0]
P = pivot(4,a,0,1)
show(P*M, [P, M])
a = -M[2,0]/M[0,0]
P = pivot(4,a,0,2)
show(P*M, [P, M])

Lorsque $i\neq j$, l'inverse de $P_{l_i \leftarrow l_i + \alpha l_j}$ n'est pas très difficile à calculer, c'est $P_{l_i \leftarrow l_i - \alpha l_j}$ :

In [5]:
PP = inv(P)
show(r"\operatorname{P}^{-1}_{l_1 \leftarrow l_1 + 3 l_0}", [inv(pivot(4, 3, 0, 1))], [pivot(4,-3,0,1)], [r"\operatorname{P}_{l_1 \leftarrow l_1 - 3 l_0}"],online=True)

Le produit de pivots travaillant sur des lignes différentes se passe aussi très bien et leur inverse n'est pas plus difficile à calculer (c'est le produit de leurs inverses qui sont eux-même des pivots donc...) :

In [6]:
MS = [pivot(4, 3, 0, 1), pivot(4, 2, 0, 2), pivot(4, 5, 0, 3)]
M = prod(*MS)
show(M, MS)
invMS = [pivot(4, -5, 0, 3), pivot(4, -2, 0, 2), pivot(4, -3, 0, 1)]
show(latex(M)+r"^{-1}", [inv(M)], invMS, [prod(*invMS)], online=True)

## 3. Récapitulons sur notre exemple

Nous avons maintenant tout ce qu'il faut pour décomposer notre matrice initiale :

In [7]:
show("A",[A])

On commence par définir $L_0$ et $U_0$ :

In [8]:
Id = identity(4)
L0 = Id
show("L_0",[L0])
U0 = A
show("U_0",[U0])

### Colonne 0

On construit d'abord la matrice $P_0$ qui annule les 3 derniers éléments de la $0$ième colonne de $U_0$ :

In [9]:
c = U0[0,0]
PS0 = [Id]
invPS0 = [Id]
if nz(c): ## On évite le cas pathologique où toute la colonne est à 0
    PS0 = [ pivot(4, -U0[i,0]/c, 0, i) for i in range(1,4) if nz(U0[i,0]) ]
    invPS0 = [ pivot(4, U0[i,0]/c, 0, i) for i in range(3,0,-1) if nz(U0[i,0]) ]
P0 = prod(Id, *PS0)
invP0 = prod(Id, *invPS0)
show(P0, PS0)
show((latex(P0) + "^{-1}"), invPS0, [invP0],online=True)

Et on multiplie $L_0$ à droite par l'inverse de $P_0$ et $U_0$ à gauche par $P_0$ :

In [10]:
show(A, [L0, "P_0^{-1}", "P_0", U0], [[L0, invP0], [P0, U0]])

On obtient les matrices $L_1$ et $U_1$ pour l'étape suivante :

In [11]:
L1=L0*invP0
show("L_1",[L1])
U1=P0*U0
show("U_1",[U1])

A noter que pour générer la matrice $L_1$ comme le produit $L_0\times P_0^{-1}$ il s'agit juste de reporter les valeurs définies pour annuler les 3 derniers éléments de la 0ième colonne de $U_0$.

### Colonne 1

On construit d'abord la matrice $P_1$ qui annule les 2 derniers éléments de la $1$ième colonne de $U_1$ :

In [12]:
c = U1[1,1]
PS1 = [Id]
invPS1 = [Id]
if nz(c): ## On évite le cas pathologique où toute la fin de colonne est à 0
    PS1 = [ pivot(4, -U1[i,1]/c, 1, i) for i in range(2,4) if nz(U1[i,1]) ]
    invPS1 = [ pivot(4, U1[i,1]/c, 1, i) for i in range(3,1,-1) if nz(U1[i,1]) ]
P1 = prod(Id, *PS1)
invP1 = prod(Id, *invPS1)
show(P1, PS1)
show((latex(P1) + "^{-1}"), invPS1, [invP1],online=True)

Et on multiplie $L_1$ à droite par l'inverse de $P_1$ et $U_1$ à gauche par $P_1$ :

In [13]:
show(A, [L1, "P_1^{-1}", "P_1", U1], [[L1, invP1], [P1, U1]])

On obtient les matrices $L_2$ et $U_2$ pour l'étape suivante :

In [14]:
L2=L1*invP1
show("L_2",[L2])
U2=P1*U1
show("U_2",[U2])

### Colonne 2

On construit d'abord la matrice $P_2$ qui annule le dernier élément de la $2$ième colonne de $U_2$ :

In [15]:
c = U2[2,2]
PS2 = [Id]
invPS2 = [Id]
if nz(c): ## On évite le cas pathologique où toute la fin de colonne est à 0
    PS2 = [ pivot(4, -U2[i,2]/c, 2, i) for i in range(3,4) if nz(U2[i,2]) ]
    invPS2 = [ pivot(4, U2[i,2]/c, 2, i) for i in range(3,2,-1) if nz(U2[i,2]) ]
P2 = prod(Id, *PS2)
invP2 = prod(Id, *invPS2)
show(P2, PS2)
show((latex(P2) + "^{-1}"), invPS2, [invP2],online=True)

Et on multiplie $L_2$ à droite par l'inverse de $P_2$ et $U_2$ à gauche par $P_2$ :

In [16]:
show(A, [L2, "P_2^{-1}", "P_2", U2], [[L2, invP2], [P2, U2]])

On obtient les matrices $L_3$ et $U_3$ c'est-à-dire $L$ et $U$ !

In [17]:
L3=L2*invP2
show("L = L_3",[L3])
U3=P2*U2
show("U = U_3",[U3])
L=L3
U=U3
B=L*U
show(A,[L,U],[B], online=True)
assert(A==B).all()

## Version algorithmique naïve

En reprenant les idées ci-dessus on obtient le code suivant. Notez qu'il s'agit d'une version naïve puisqu'elle effectue de nombreuses mutiplications inutiles : on peut remplir la matrice $L$ directement en allant modifier ses coefficients et mettre à jour $U$ en effectuant explicitement les pivots sur les lignes concernées.

In [18]:
def naiveLU(A, trace=False):
    "décompose la matrice A en un produit LU par pivot de Gauss"
    n = A.shape[0]
    I = identity(n)
    L, U = I, A
    for k in range(n-1):
        c = U[k,k]
        if z(c):
            continue
        PS = [ pivot(n, -U[i,k]/c, k, i) for i in range(k+1,n) if nz(U[i,k]) ]
        invPS = [ pivot(n, U[i,k]/c, k, i) for i in range(n-1,k,-1) if nz(U[i,k]) ]
        P = prod(I, *PS)
        invP = prod(I, *invPS)
        L, U = L*invP, P*U
        if trace:
            show("P_{}".format(k+1), [P])
            show("P^{}_{}".format("{-1}",k+1), [invP])
            show("L_{}".format(k+1), [L])
            show("U_{}".format(k+1), [U])
    return L, U

In [19]:
LA = [ rmat(8) for i in range(10) ]
for A in LA:
    show(A, list(naiveLU(A)))

## 4. Éviter de faire trop de calculs

Voici enfin le même code en effectuant les pivots directement en manipulant les matrices L et U.

In [20]:
def decompLU(A, trace=False):
    "décompose la matrice A en un produit LU par pivot de Gauss"
    n = A.shape[0]
    I = identity(n)
    L, U = I, A.copy()
    for k in range(n-1):
        c = U[k,k]
        if z(c):
            continue
        for i in range(k+1,n):
            a = U[i,k]/c
            if nz(a):
                L[i,k] = a
                for j in range(k,n):
                    U[i,j] -= a*U[k,j]
        if trace:
            show("L_{}".format(k+1), [L])
            show("U_{}".format(k+1), [U])
    return L, U

In [21]:
for A in LA:
    show(A,list(decompLU(A)))

## 5. La factorisation LU par blocs


Dans cette version la factorisation LU précédente va être utilisée pour calculer la factorisation LU des blocs diagonaux.

A partir de l'exemple suivant où la matrice $A$ est de taille $12 \times 12$ et en considérant $l=4$ pour la taille des blocs on peut exprimer la factorisation LU de la manière suivante
$$\begin{pmatrix}
  A_{00}   &A_{01} &A_{02}\\
  A_{10}   &A_{11} &A_{12}\\
  A_{20}   &A_{21} &A_{22}\\
\end{pmatrix} = \begin{pmatrix}
  L_{00}   &0 &0 \\
  L_{10}   &L_{11} &0\\
  L_{20}   &L_{21} &L_{22}\\
\end{pmatrix} 
\times  \begin{pmatrix}
  U_{00}   &U_{01} &U_{02}\\
  0  &U_{11} &U_{12}\\
  0 &0 &U_{22}\\
\end{pmatrix}$$
 
1. $L_{00}$ et $U_{00}$ sont le résultat de la factorisation LU de $A_{00}$ 

In [22]:
n=12
l=4
b=3
A=rmat(n)
show(A)

In [23]:
A00=A[0:l,0:l]
[L00,U00]=decompLU(A00,0)
show("A_{00}",[A00])
show("L_{00}U_{00}",[L00,U00],[L00*U00])

2. Mais en regardant le résultat qu'on veut obtenir avec la première colonne de $L$ et la première ligne de $U$

$\begin{pmatrix}
  L_{00}   &0 &0\\
  L_{10}   &. &.\\
  L_{20}   &. &.\\
\end{pmatrix} 
\times  \begin{pmatrix}
  U_{00}   &U_{01} &U_{02}\\
  0  &. &.\\
  0 &. &.\\
\end{pmatrix}= \begin{pmatrix}
L_{00}U_{00} &L_{00}U_{01} &L_{00}U_{02}\\
L_{10}U_{00} &. &.\\
L_{20}U_{00} &. &.\\
\end{pmatrix}=\begin{pmatrix}
  A_{00}   &A_{01} &A_{02}\\
  A_{10}   &A_{11} &A_{12}\\
  A_{20}   &A_{21} &A_{22}\\
\end{pmatrix}$

On voit qu'on peut calculer $L_{i0}$ et $U_{0j}$ grâce aux relations 

$L_{i0}U_{00}= A_{i0}$ et $L_{00}U_{0j}=A_{0j}$

En effet les matrices $L_{00}$ et $U_{00}$ sont respectivement triangulaire inférieure et triangulaire supérieure et une résolution de système permet de résoudre les 2 équations.

In [24]:
def resolutionSup(U,A):
    n = A.shape[0]
    L=zeros((n,n))
    for i in range(n):
        for j in range(n):
            tmp = 0
            for k in range(j):
                tmp += L[i,k]*U[k,j] 
            L[i,j] = (A[i,j]-tmp)/U[j,j]
    return L

def resolutionInf(L,A):
    n = A.shape[0]
    U=zeros((n,n))
    for j in range(n):
        for i in range(n):
            tmp = 0
            for k in range(i):
                tmp += U[k,j]*L[i,k] 
            U[i,j] = (A[i,j]-tmp)/L[j,j]
    return U

def subm(M,k,i,j): 
    return M[k*i:k*(i+1),k*j:k*(j+1)]


A10=subm(A,l,1,0)
show("A_{10}",[A10])
L10=resolutionSup(U00,A10)
show("U_{00}",[U00])
show("L_{10}",[L10])
show("L_{10}U_{00}",[L10*U00])

A01=subm(A,l,0,1)
show("A_{01}",[A01])
U01=resolutionInf(L00,A01)
show("U_{01}",[U01])
show("L_{00}U_{01}",[L00*U01])

In [25]:
A20=subm(A,l,2,0)
show("A_{20}",[A20])
L20=resolutionSup(U00,A20)
show("L_{20}",[L20])
show("L_{20}U_{00}",[L20*U00])
A02=subm(A,l,0,2)
show("A_{02}",[A02])
U02=resolutionInf(L00,A02)
show("U_{02}",[U02])
show("L_{00}U_{02}",[L00*U02])

3. Enfin si on écrit 
$\begin{pmatrix}
  L_{00}   &0 &0\\
  L_{10}   &I &0\\
  L_{20}   &0 &I\\
\end{pmatrix} 
\times  \begin{pmatrix}
  U_{00}   &U_{01} &U_{02}\\
  0  &\tilde A_{11} &\tilde A_{12}\\
  0  &\tilde A_{21} &\tilde A_{22}\\
\end{pmatrix}$

on a alors la dernière équation $A_{ij}=L_{i0}U_{0j}+\tilde A_{ij}$ qui permet de calculer $\tilde A_{ij}$ pour $i,j>0$

4. Il reste à poursuivre en calculant la factorisation LU de la matrice $\tilde A$. En effet, si on écrit

$\begin{pmatrix}
  \tilde A_{11} &\tilde A_{12}\\
  \tilde A_{21} &\tilde A_{22}\\
\end{pmatrix}=\begin{pmatrix}
  L_{11} &0\\
  L_{21} &L_{22}\\
\end{pmatrix}\times \begin{pmatrix}
  U_{11} &U_{12}\\
   0 &U_{22}\\
\end{pmatrix}=\begin{pmatrix}
  L_{11}U_{11} &L_{11}U_{12}\\
  L_{21}U_{11} &L_{21}U_{12}+L_{22}U_{22}\\
\end{pmatrix}$

On a 

$\begin{pmatrix}
  L_{00}   &0 &0 \\
  L_{10}   &L_{11} &0\\
  L_{20}   &L_{21} &L_{22}\\
\end{pmatrix} 
\times  \begin{pmatrix}
  U_{00}   &U_{01} &U_{02}\\
  0  &U_{11} &U_{12}\\
  0 &0 &U_{22}\\
\end{pmatrix}=\begin{pmatrix}
L_{00}U_{00}   &L_{00}U_{01} &L_{00}U_{02} \\
  L_{10}U_{00}   &L_{10}U_{01}+L_{11}U_{11} &L_{10}U_{02}+L_{11}U_{12}\\
  L_{20}U_{00}   &L_{20}U_{01}+L_{21}U_{11} &L_{20}U_{02}+L_{21}U_{12}+L_{22}U_{22}\\
  \end{pmatrix}=\begin{pmatrix}
  A_{00}   &A_{01} &A_{02}\\
  A_{10}   &L_{10}U_{01}+\tilde A_{11} &L_{10}U_{02}+\tilde A_{12}\\
  A_{20}   &L_{20}U_{01}+\tilde A_{21} &L_{20}U_{02}+\tilde A_{22}\\
\end{pmatrix}=\begin{pmatrix}
  A_{00}   &A_{01} &A_{02}\\
  A_{10}   &A_{11} &A_{12}\\
  A_{20}   &A_{21} &A_{22}\\
\end{pmatrix}$


In [26]:
def DecompLUBlocs(A,n,l,b):
    U = zeros((n,n))
    L = zeros((n,n))
    for e in range(b):
        Aee=subm(A,l,e,e)
        [Lee,Uee]=decompLU(Aee)
        subm(L,l,e,e)[:]=Lee
        subm(U,l,e,e)[:]=Uee     
        for j in range(b-e):
            j+=e;
            Aje=subm(A,l,j,e)
            Lje=resolutionSup(Uee,Aje)
            subm(L,l,j,e)[:]=Lje
            Aej=subm(A,l,e,j)
            Uej=resolutionInf(Lee,Aej)
            subm(U,l,e,j)[:]=Uej
        for i in range(b-e):
            i+=e
            for j in range(b-e):
                j+=e
                Aij=subm(A,l,i,j)
                Lie=subm(L,l,i,e)
                Uej=subm(U,l,e,j)
                Atildeij=Aij-Lie*Uej
                subm(A,l,i,j)[:]=Atildeij
        print("e=",e)
        show("L",[L])
        show("U",[U])
    return L, U

show("A",[A])
[L,U]=DecompLUBlocs(A,n,l,b)
show("LU",[L*U])

e= 0


e= 1


e= 2
